# **1\. CREATING INITIAL TABLES**

In [75]:
DROP TABLE CourseProfessors, MandatoryCourseStudies, Courses, Studies, Professors

CREATE TABLE Professors(
    profCode INT PRIMARY KEY,
    firstName NVARCHAR(50) COLLATE Cyrillic_General_CI_AS, 
    lastName NVARCHAR(50) COLLATE Cyrillic_General_CI_AS,
    email NVARCHAR(50),
    numberOfCourses INT
);

CREATE TABLE Studies(
    studiesCode INT PRIMARY KEY,
    name NVARCHAR(70) COLLATE Cyrillic_General_CI_AS,
    yearsToComplete NVARCHAR(15),
);

CREATE TABLE Courses(
    coursesCode NVARCHAR(10) PRIMARY KEY,
    name NVARCHAR(70) COLLATE Cyrillic_General_CI_AS,
);

CREATE TABLE MandatoryCourseStudies(
    studiesCode INT,
    coursesCode NVARCHAR(10),
    semester INT,
    CONSTRAINT CourseStudies_PK PRIMARY KEY(coursesCode, studiesCode),
    CONSTRAINT SCourseCode_FK FOREIGN KEY (coursesCode) REFERENCES Courses(coursesCode),
    CONSTRAINT SStudiesCode_FK FOREIGN KEY (studiesCode) REFERENCES Studies(studiesCode),
)

CREATE TABLE CourseProfessors(
    coursesCode NVARCHAR(10),
    profCode INT,
    CONSTRAINT CourseProfessors_PK PRIMARY KEY(coursesCode, profCode),
    CONSTRAINT PCourseCode_FK FOREIGN KEY (coursesCode) REFERENCES Courses(coursesCode),
    CONSTRAINT PStudiesCode_FK FOREIGN KEY (profCode) REFERENCES Professors(profCode),
)

Commands completed successfully.

Total execution time: 00:00:00.044

# **2\. POPULATING THE TABLES**

In [76]:
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (1, N'Марјан', N'Гушев', 'marjan.gushev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (2, N'Катерина', N'Здравкова', 'katerina.zdravkova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (3, N'Сузана', N'Лошковска', 'suzana.loshkovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (4, N'Коста', N'Митрески', 'kosta.mitreski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (5, N'Верица', N'Бакева', 'verica.bakeva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (6, N'Владимир', N'Трајковиќ', 'vladimir.trajkovik@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (7, N'Ана', N'Мадевска Богданова', 'ana.madevska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (8, N'Дејан', N'Ѓорѓевиќ', 'dejan.gjorgjevikj@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (9, N'Димитар', N'Трајанов', 'dimitar.trajanov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (10, N'Андреа', N'Кулаков', 'andrea.kulakov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (11, N'Љупчо', N'Антовски', 'ljupcho.antovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (12, N'Марија', N'Михова', 'marija.mihova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (13, N'Слободан', N'Калајџиски', 'slobodan.kalajdziski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (14, N'Невена', N'Ацковска', 'nevena.ackovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (15, N'Горан', N'Велинов', 'goran.velinov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (16, N'Анастас', N'Мишев', 'anastas.mishev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (17, N'Соња', N'Филипоска', 'sonja.filiposka@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (18, N'Иван', N'Чорбев', 'ivan.chorbev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (19, N'Ласко', N'Баснарков', 'lasko.basnarkov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (20, N'Боро', N'Јакимовски', 'boro.jakimovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (21, N'Весна', N'Димитрова', 'vesna.dimitrova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (22, N'Гоце', N'Арменски', 'goce.armenski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (23, N'Соња', N'Гиевска', 'sonja.gievska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (24, N'Дејан', N'Спасов', 'dejan.spasov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (25, N'Ивица', N'Димитровски', 'ivica.dimitrovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (26, N'Игор', N'Мишковски', 'igor.mishkovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (27, N'Ѓорѓи', N'Маџаров', 'gjorgji.madjarov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (28, N'Смилка', N'Јанеска-Саркањац', 'smilka.janeska.sarkanjac@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (29, N'Вангел', N'Ајановски', 'vangel.ajanovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (30, N'Весна', N'Димитриевска Ристовска', 'vesna.dimitrievska.ristovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (31, N'Миле', N'Јованов', 'mile.jovanov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (32, N'Билјана', N'Стојкоска', 'biljana.stojkoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (33, N'Кире', N'Триводалиев', 'kire.trivodaliev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (34, N'Сашо', N'Граматиков', 'sasho.gramatikov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (35, N'Мирослав', N'Мирчев', 'miroslav.mirchev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (36, N'Георгина', N'Мирчева', 'georgina.mircheva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (37, N'Магдалена', N'Костоска', 'magdalena.kostoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (38, N'Александра', N'Поповска Митровиќ', 'aleksandra.popovska.mitrovikj@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (39, N'Наташа', N'Илиевска', 'natasha.ilievska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (40, N'Билјана', N'Тојтовска', 'biljana.tojtovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (41, N'Милош', N'Јовановиќ', 'milos.jovanovik@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (42, N'Андреја', N'Наумоски', 'andreja.naumoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (43, N'Панче', N'Рибарски', 'pance.ribarski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (44, N'Христина', N'Михајлоска', 'hristina.mihajloska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (45, N'Иван', N'Китановски', 'ivan.kitanovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (46, N'Ефтим', N'Здравевски', 'eftim.zdravevski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (47, N'Петре', N'Ламески', 'petre.lameski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (48, N'Катарина', N'Тројачанец Динева', 'katarina.trojacanec@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (49, N'Владимир', N'Здравески', 'vladimir.zdraveski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (50, N'Бојана', N'Котеска', 'bojana.koteska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (51, N'Моника', N'Симјаноска', 'monika.simjanoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (52, N'Методија', N'Јанчески', 'metodija.jancheski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (53, N'Ристе', N'Стојанов', 'riste.stojanov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (54, N'Александра', N'Дединец', 'aleksandra.dedinec@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (55, N'Емил', N'Станков', 'emil.stankov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (56, N'Борис', N'Станоев', 'boris.stanoev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (57, N'Бобан', N'Јоксимоски', 'boban.joksimoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (58, N'Илинка', N'Иваноска', 'ilinka.ivanoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (59, N'Бојан', N'Илијоски', 'bojan.ilijoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (60, N'Костадин', N'Мишев', 'kostadin.mishev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (61, N'Александар', N'Тенев', 'aleksandar.tenev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (62, N'Александар', N'Стојменски', 'aleksandar.stojmenski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (63, N'Ненад', N'Анчев', 'nenad.anchev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (64, N'Војдан', N'Ќорвезироски', 'vojdan.kjorveziroski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (65, N'Мартина', N'Тошевска', 'martina.toshevska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (66, N'Влатко', N'Спасев', 'vlatko.spasev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (67, N'Петар', N'Секулоски', 'petar.sekuloski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (68, N'Стефан', N'Андонов', 'stefan.andonov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (69, N'Кирил', N'Ќироски', 'kiril.kjiroski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (70, N'Ана', N'Тодоровска', 'ana.todorovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (71, N'Јована', N'Добрева', 'jovana.dobreva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (72, N'Јана', N'Кузманова', 'jana.kuzmanova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (73, N'Димитар', N'Китановски', 'dimitar.kitanovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (74, N'Томе', N'Ефтимов', 'tome.eftimov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (75, N'Андреј', N'Ристески', 'andrej.risteski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (76, N'Славе', N'Темков', 'slave.temkov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (77, N'Милена', N'Трајаноска', 'milena.trajanoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (78, N'Живко', N'Атанаскоски', 'zhivko.atanaskoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (79, N'Димитар', N'Милески', 'dimitar.mileski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (80, N'Зорица', N'Карапанчева', 'zorica.karapancheva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (81, N'Марија', N'Танеска', 'marija.taneska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (82, N'Мила', N'Додевска', 'mila.dodevska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (83, N'Ана', N'Атанасова', 'ana.atanasova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (84, N'Александар', N'Петрушев', 'aleksandar.petrushev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (85, N'Мартин', N'Динев', 'martin.dinev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (86, N'Владислав', N'Бидиков', 'vladislav.bidikov@finki.ukim.mk');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.099

In [77]:
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (0, N'Интернет, мрежи и безбедност', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (1, N'Интернет, мрежи и безбедност', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (2, N'Компјутерска едукација', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (3, N'Компјутерски науки', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (4, N'Компјутерски науки', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (5, N'Компјутерско инженерство', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (6, N'Компјутерско инженерство', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (7, N'Примена на информациски технологии', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (8, N'Примена на информациски технологии', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (9, N'Софтверско инженерство и информациски технологии', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (10, N'Софтверско инженерство и информациски технологии', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (11, 'Software engineering and information systems', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (12, 'Software engineering and information systems', '4');

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.047

In [78]:
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W005', N'Бизнис и менаџмент');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W007', N'Вовед во компјутерските науки');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S013', N'Калкулус');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W018', N'Професионални вештини');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W020', N'Структурно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S003', N'Архитектура и организација на компјутери');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W011', N'Дискретна математика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S015', N'Објектно ориентирана анализа и дизајн');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S016', N'Објектно-ориентирано програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('0', N'Спорт и здравје');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S034', N'Калкулус 2');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S116', N'Компјутерски компоненти');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S120', N'Креативни вештини за решавање проблеми');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S026', N'Маркетинг');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S146', N'Основи на веб дизајн');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S066', N'Основи на сајбер безбедноста');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W001', N'Алгоритми и податочни структури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W006', N'Веројатност и статистика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W014', N'Компјутерски мрежи и безбедност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S100', N'Економија за ИКТ инженери');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W104', N'Инженерска математика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W109', N'Интернет програмирање на клиентска страна');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W140', N'Напредно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W165', N'Управување со техничката поддршка');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W167', N'Шаблони за дизајн на кориснички интерфејси');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S002', N'Анализа на софтверските барања');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S017', N'Оперативни системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S061', N'Безжични и мобилни системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S030', N'Вештачка интелигенција');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S082', N'Визуелно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S084', N'Вовед во екоинформатиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S090', N'Вовед во случајни процеси');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S095', N'Дигитално процесирање на слика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S097', N'Дизајн на алгоритми');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S099', N'Е-влада');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S042', N'Електрични кола');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S110', N'Интернет технологии');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S114', N'Компјутерска графика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S119', N'Концепти на информатичко општество');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S124', N'Медиуми и комуникации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S164', N'Теорија на информации со дигитални комуникации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W004', N'Бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W008', N'Вовед во науката за податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W009', N'Дизајн и архитектура на софтвер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W060', N'Администрација на системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W024', N'Веб програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W081', N'Визуелизација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W043', N'Информациска безбедност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W053', N'Компјутерска етика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W115', N'Компјутерски звук, говор и музика ');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W035', N'Линеарна алгебра и примени');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W134', N'Мултимедиски мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W136', N'Напреден веб дизајн');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W148', N'Основи на роботиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S010', N'Дизајн на интеракцијата човек-компјутер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S012', N'Интегрирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S019', N'Софтверски квалитет и тестирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S073', N'Агентно-базирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S077', N'Безжични мултимедиски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S040', N'Вградливи микропроцесорски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S062', N'Виртуелизација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S087', N'Вовед во мрежна наука');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S091', N'Географски информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S093', N'Дигитална форензика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S025', N'Електронска и мобилна трговија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S118', N'Континуирана интеграција и испорака');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S036', N'Машинско учење');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S135', N'Мултимедиски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S149', N'Паралелно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S150', N'Податочно рударство');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S047', N'Процесирање на сигналите');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S155', N'Сервисно ориентирани архитектури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S159', N'Софтверски дефинирана безбедност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S163', N'Статистичко моделирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S039', N'Формални јазици и автомати');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W021', N'Тимски проект');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W072', N'Автономна роботика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W074', N'Администрација на бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W075', N'Анализа и дизајн на ИС');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W079', N'Веб базирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W085', N'Вовед во биоинформатиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W088', N'Вовед во паметни градови');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W089', N'Вовед во препознавање на облици');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W092', N'Дигитална постпродукција');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W103', N'Имплементација на системи со слободен и отворен код');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W105', N'Иновации во ИКТ');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W108', N'Интернет на нештата');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W123', N'Машинска визија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W027', N'Менаџмент на информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W126', N'Методологија на истражувањето во ИКТ');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W128', N'Мобилни информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W129', N'Мобилни платформи и програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W137', N'Напредна интеракција човек компјутер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W142', N'Обработка на природните јазици');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W144', N'Операциони истражувања');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W068', N'Пресметување во облак');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W152', N'Програмирање на видео игри');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W038', N'Програмски парадигми');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W154', N'Рударење на масивни податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W156', N'Системи за поддршка при одлучувањето');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W048', N'Софтвер за вградливи системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W160', N'Софтверски дефинирани мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W161', N'Социјални мрежи и медиуми');
INSERT INTO Courses(coursesCode, name) VALUES ('1', N'Дипломска работа');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S022', N'Управување со ИКТ проекти');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S07', N'Биолошки инспирирано пресметување');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S121', N'Блоковски вериги и криптовалути');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S080', N'Веб пребарувачки системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S083', N'Виртуелна реалност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S086', N'Вовед во когнитивни науки');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S102', N'ИКТ за развој');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S106', N'Интелигентни информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S107', N'Интелигентни системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S113', N'Компјутерска анимација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S127', N'Мобилни апликации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S130', N'Моделирање и менаџирање на бизнис процеси');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S132', N'Модерни трендови во роботика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S138', N'Напредни бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S141', N'Неструктурирани бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S028', N'Претприемништво');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S157', N'Складови на податоци и аналитичка обработка');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S162', N'Споделување и пресметување во толпа');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W070', N'Педагогија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S052', N'ИТ системи за учење');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S045', N'Компјутерски архитектури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S023', N'Бизнис статистика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W055', N'Мултимедијални технологии');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W096', N'Дигитизација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W050', N'Дизајн на образовен софтвер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S056', N'Персонализирано учење');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S071', N'Психологија на училишна возраст');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S057', N'Работа со надарени ученици');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S094', N'Дигитални библиотеки');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S166', N'Учење на далечина');
INSERT INTO Courses(coursesCode, name) VALUES ('2', N'Самостоен проект');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S069', N'Македонски јазик');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S054', N'Методика на информатиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W031', N'Дискретни структури 1');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S032', N'Дискретни структури 2');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W037', N'Паралелно и дистрибуирано процесирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S125', N'Мерење и анализа на интернет сообраќај');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S153', N'Процесна роботика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S158', N'Современи компјутерски архитектури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W145', N'Оптички мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S151', N'Пресметковна биологија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S112', N'Програмски јазици и компајлери');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W049', N'Физика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W147', N'Основи на комуникациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S143', N'Одржливи и енергетски ефикасни компјутерски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W044', N'Компјутерска електроника');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W131', N'Моделирање и симулација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S101', N'Етичко хакирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W046', N'Компјутерски мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W067', N'Основи на теоријата на информации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S029', N'Софтверско инженерство');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W033', N'Калкулус 1');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W041', N'Дизајн на дигитални кола');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S051', N'ИКТ во образованието');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S063', N'Дизајн на компјутерски мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W064', N'Дистрибуирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S059', N'Администрација на мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W065', N'Мрежна безбедност');


(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.135

In [79]:
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S066', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2W067', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2S061', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L3W060', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L3W065', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S066', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2W067', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2S061', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W060', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W065', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3S059', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3S062', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W064', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W068', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, '1', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3S063', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W011', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W070', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S052', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S013', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W055', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2S051', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W050', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W053', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S056', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S071', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S057', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, '2', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, '1', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S069', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S054', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W031', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S032', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2S030', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L3S036', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W031', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S032', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2S030', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W035', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W037', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3S036', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3S039', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W038', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, '1', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W041', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W049', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2S042', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L3S040', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W041', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W049', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2S042', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W043', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W044', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3S040', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3S047', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W048', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3S022', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, '1', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W011', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S023', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S026', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L3W024', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W011', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S023', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S026', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W024', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W008', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S025', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W027', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W021', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, '1', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S028', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S022', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W008', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S012', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W021', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S022', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, '1', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W008', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S012', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W021', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S022', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, '1', 8);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.208

In [80]:
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W005', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W005', 28);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 3);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 42);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 25);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S045', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 25);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S066', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S066', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W046', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W046', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W046', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W067', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W067', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W067', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S061', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S061', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W060', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W060', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 15);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 46);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 29);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W065', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W065', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S059', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S062', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W064', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W064', 47);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W064', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W068', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S054', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S052', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W055', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S051', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S051', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W050', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W053', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S056', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W031', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W031', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W033', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 10);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 47);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 25);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 46);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 3);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S036', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S036', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S039', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W038', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W038', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W038', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W041', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W041', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W049', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S116', 42);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S116', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S120', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S120', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 42);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 28);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S066', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S042', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W043', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W044', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S040', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S040', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S047', 47);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S047', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W048', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W048', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W048', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S028', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 28);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 15);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W024', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W024', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W024', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S015', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S015', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S015', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S002', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S002', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S002', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W009', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W009', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W009', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S012', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S019', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S019', 44);

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.147

# **3\. TABLE RESULTS VISUELIZATION**

In [81]:
SELECT * FROM Professors

(86 rows affected)

Total execution time: 00:00:00.068

profCode,firstName,lastName,email,numberOfCourses
1,Марјан,Гушев,marjan.gushev@finki.ukim.mk,NULL
2,Катерина,Здравкова,katerina.zdravkova@finki.ukim.mk,NULL
3,Сузана,Лошковска,suzana.loshkovska@finki.ukim.mk,NULL
4,Коста,Митрески,kosta.mitreski@finki.ukim.mk,NULL
5,Верица,Бакева,verica.bakeva@finki.ukim.mk,NULL
6,Владимир,Трајковиќ,vladimir.trajkovik@finki.ukim.mk,NULL
7,Ана,Мадевска Богданова,ana.madevska@finki.ukim.mk,NULL
8,Дејан,Ѓорѓевиќ,dejan.gjorgjevikj@finki.ukim.mk,NULL
9,Димитар,Трајанов,dimitar.trajanov@finki.ukim.mk,NULL
10,Андреа,Кулаков,andrea.kulakov@finki.ukim.mk,NULL


In [82]:
SELECT * FROM Studies

(13 rows affected)

Total execution time: 00:00:00.026

studiesCode,name,yearsToComplete
0,"Интернет, мрежи и безбедност",3
1,"Интернет, мрежи и безбедност",4
2,Компјутерска едукација,4
3,Компјутерски науки,3
4,Компјутерски науки,4
5,Компјутерско инженерство,3
6,Компјутерско инженерство,4
7,Примена на информациски технологии,3
8,Примена на информациски технологии,4
9,Софтверско инженерство и информациски технологии,3


In [83]:
SELECT * FROM MandatoryCourseStudies

(282 rows affected)

Total execution time: 00:00:00.068

studiesCode,coursesCode,semester
0,0,2
1,0,2
2,0,2
3,0,2
4,0,2
5,0,2
6,0,2
7,0,2
8,0,2
9,0,2


In [84]:
SELECT * FROM CourseProfessors

(216 rows affected)

Total execution time: 00:00:00.067

coursesCode,profCode
F18L1S003,1
F18L1S003,11
F18L1S003,24
F18L1S003,26
F18L1S003,32
F18L1S003,34
F18L1S003,35
F18L1S003,37
F18L1S003,49
F18L1S003,54


# **4\. TABLE MANIPULATION**

> ## 4.1. Setting the correct number of mandatory courses eash professor is involved in (based on the information listed in the faculty's webpage)

In [91]:
UPDATE Professors
SET numberOfCourses = ProfessorCount
FROM (
    SELECT ProfCode, COUNT(*) AS ProfessorCount
    FROM CourseProfessors
    GROUP BY ProfCode
) AS Counts
WHERE Professors.profCode = Counts.ProfCode;

UPDATE Professors
SET numberOfCourses = 0
WHERE Professors.numberOfCourses IS NULL

SELECT * FROM Professors

(49 rows affected)

(0 rows affected)

(86 rows affected)

Total execution time: 00:00:00.083

profCode,firstName,lastName,email,numberOfCourses
1,Марјан,Гушев,marjan.gushev@finki.ukim.mk,4
2,Катерина,Здравкова,katerina.zdravkova@finki.ukim.mk,4
3,Сузана,Лошковска,suzana.loshkovska@finki.ukim.mk,2
4,Коста,Митрески,kosta.mitreski@finki.ukim.mk,4
5,Верица,Бакева,verica.bakeva@finki.ukim.mk,6
6,Владимир,Трајковиќ,vladimir.trajkovik@finki.ukim.mk,4
7,Ана,Мадевска Богданова,ana.madevska@finki.ukim.mk,5
8,Дејан,Ѓорѓевиќ,dejan.gjorgjevikj@finki.ukim.mk,4
9,Димитар,Трајанов,dimitar.trajanov@finki.ukim.mk,4
10,Андреа,Кулаков,andrea.kulakov@finki.ukim.mk,1


> ## 4.2. Adding new tables with different statistics calculated based on the other tables' values

In [125]:
DROP TABLE ProfessorsStatistics, CourseStatistics, StudiesStatistics

CREATE TABLE ProfessorsStatistics (
    calculationSequenceCode INT PRIMARY KEY,
    totalNumberOfProfessors INT,
    avgNumOfMandatoryCoursesPerProfessor FLOAT,
);

CREATE TABLE CourseStatistics (
    calculationSequenceCode INT PRIMARY KEY,
    totalNumberOfCourses INT,
    mandatoryCoursesCount INT,
    electiveCoursesCount INT,
)

CREATE TABLE StudiesStatistics (
    calculationSequenceCode INT PRIMARY KEY,
    totalNumberOfStudies INT,
    avgNumMandatoryCoursesPerSemester FLOAT,
    totalNumberOfThreeYearStudies INT,
    totalNumberOfFourYearStudies INT
)

Commands completed successfully.

Total execution time: 00:00:00.072

In [126]:
DECLARE @Count INT;
DECLARE @profCount INT;
DECLARE @avgMandatoryCourses INT;
DECLARE @max INT;

SELECT @Count = COUNT(*)
FROM ProfessorsStatistics;

SELECT @max = MAX(calculationSequenceCode) 
FROM ProfessorsStatistics;

SELECT @profCount = totalNumberOfProfessors,
       @avgMandatoryCourses = avgNumOfMandatoryCoursesPerProfessor
FROM ProfessorsStatistics
WHERE calculationSequenceCode = @max;

IF (@Count = 0)
BEGIN
    INSERT INTO ProfessorsStatistics (calculationSequenceCode) VALUES (0);
END
ELSE IF (@Count != 0 AND @profCount IS NOT NULL AND @avgMandatoryCourses IS NOT NULL)
BEGIN
    INSERT INTO ProfessorsStatistics (calculationSequenceCode) VALUES (@max + 1);
END;

(1 row affected)

Total execution time: 00:00:00.010

In [127]:
DECLARE @max INT;

SELECT @max = MAX(calculationSequenceCode) 
FROM ProfessorsStatistics;

UPDATE ProfessorsStatistics 
SET avgNumOfMandatoryCoursesPerProfessor = (SELECT AVG(numberOfCourses) FROM Professors),
    totalNumberOfProfessors = (SELECT COUNT(profCode) FROM Professors)
WHERE calculationSequenceCode = @max;

SELECT * FROM ProfessorsStatistics;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.072

calculationSequenceCode,totalNumberOfProfessors,avgNumOfMandatoryCoursesPerProfessor
0,86,2


In [129]:
DECLARE @count INT;
DECLARE @lastCalculationSeqCode INT;
DECLARE @totalNumberOfCourses INT;
DECLARE @mandatoryCoursesCount INT;
DECLARE @electiveCoursesCount INT;

SELECT @Count = COUNT(*)
FROM CourseStatistics;

SELECT @lastCalculationSeqCode = MAX(calculationSequenceCode) 
FROM CourseStatistics;

SELECT @totalNumberOfCourses = totalNumberOfCourses,
       @mandatoryCoursesCount = mandatoryCoursesCount,
       @electiveCoursesCount = electiveCoursesCount
FROM CourseStatistics
WHERE calculationSequenceCode = @lastCalculationSeqCode;

IF (@Count = 0)
BEGIN
    INSERT INTO CourseStatistics (calculationSequenceCode) VALUES (0);
END
ELSE IF (@Count != 0 AND @totalNumberOfCourses IS NOT NULL AND @mandatoryCoursesCount IS NOT NULL AND @electiveCoursesCount IS NOT NULL)
BEGIN
    INSERT INTO CourseStatistics (calculationSequenceCode) VALUES (@lastCalculationSeqCode + 1);
END;

Commands completed successfully.

Total execution time: 00:00:00.048

In [ ]:
DECLARE @lastCalculationSeqCode INT;

SELECT @lastCalculationSeqCode = MAX(calculationSequenceCode) 
FROM CourseStatistics;

UPDATE CourseStatistics 
SET totalNumberOfCourses = (SELECT COUNT(coursesCode) FROM Courses),
    mandatoryCoursesCount = (SELECT COUNT(profCode) FROM Professors)
WHERE calculationSequenceCode = @max;
SELECT * FROM ProfessorsStatistics;